In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import Row,Column
import pandas as pd
import numpy as np
import time

from pyspark.sql.functions import when,log,exp

spark = SparkSession.builder \
    .master("local[2]") \
    .appName("COM6012 Assignment Task2") \
    .config("spark.driver.memory", "8g")\
    .config("spark.executor.cores", 8)\
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("OFF")

In [2]:
#start Data pre-processing
raw_df = spark.read.csv('./Dataset/ClaimPredictionChallenge/train_set.csv',header= True)

In [3]:
raw_df =raw_df.select('Vehicle','Var1','Var2','Var3','Var4','Var5','Var6','Var7','Var8','NVVar1','NVVar2','NVVar3','NVVar4',
                  'Cat1','Cat2','Cat3','Cat4','Cat5','Cat6','Cat7','Cat8','Cat9','Cat10','Cat11','Cat12','Calendar_Year','Model_Year','Claim_Amount')

In [4]:
for col in raw_df.columns:
    raw_df = raw_df.filter((raw_df[col] != '?'))

In [5]:
from pyspark.sql.types import DoubleType
from pyspark.sql.types import IntegerType

double_data = ['Var1','Var2','Var3','Var4','Var5','Var6','Var7','Var8','NVVar1','NVVar2','NVVar3','NVVar4']
int_data = ['Vehicle','Calendar_Year','Model_Year','Claim_Amount']
input_features = double_data+int_data
for col in double_data:
    raw_df = raw_df.withColumn(col, raw_df[col].cast(DoubleType()))
for col in int_data:
    raw_df = raw_df.withColumn(col, raw_df[col].cast(IntegerType()))

In [6]:
raw_df = raw_df.withColumn(int_data[0], (raw_df[int_data[0]]-2005))
raw_df = raw_df.withColumn(int_data[1], (raw_df[int_data[1]]-1981))

In [7]:
categorical_features = {'Cat1':11,'Cat2':4,'Cat3':7,'Cat4':4,'Cat5':4,'Cat6':7,'Cat7':5,'Cat8':4,'Cat9':2,'Cat10':4,'Cat11':7,'Cat12':7}

In [8]:
from pyspark.ml.feature import StringIndexer,OneHotEncoderEstimator

for col,num in categorical_features.items():
    name = col+'_id'
    indexer = StringIndexer(inputCol=col, outputCol=name)
    trainingData = indexer.fit(trainingData).transform(trainingData)
    testData = indexer.fit(testData).transform(testData)

NameError: name 'trainingData' is not defined

In [ ]:
raw_df = raw_df.select('Var1','Var2','Var3','Var4','Var5','Var6','Var7','Var8',\
                        'NVVar1','NVVar2','NVVar3','NVVar4',\
                        'Cat1_id','Cat2_id','Cat3_id','Cat4_id','Cat5_id','Cat6_id','Cat7_id','Cat8_id','Cat9_id','Cat10_id','Cat11_id','Cat12_id',\
                        'Calendar_Year','Model_Year','Claim_Amount')

In [ ]:
category_id = ['Cat1_id','Cat2_id','Cat3_id','Cat4_id','Cat5_id','Cat6_id','Cat7_id','Cat8_id','Cat9_id','Cat10_id','Cat11_id','Cat12_id']

In [ ]:
cat_ohe = []
for col in category_id:
    cat_ = col.replace('_id','_ohe')
    cat_ohe.append(cat_)
    input_features.append(cat_)
    

In [ ]:
data = raw_df.select('Var1','Var2','Var3','Var4','Var5','Var6','Var7','Var8','NVVar1','NVVar2','NVVar3','NVVar4',
                  'Cat1_id','Cat2_id','Cat3_id','Cat4_id','Cat5_id','Cat6_id','Cat7_id','Cat8_id','Cat9_id','Cat10_id','Cat11_id','Cat12_id','Calendar_Year','Model_Year','Claim_Amount')

encoder = OneHotEncoderEstimator(inputCols=category_id, outputCols=cat_ohe)

encoder_data = encoder.fit(data)
data_ohe = encoder_data.transform(data)


In [14]:
#assemble cat features
from pyspark.ml.feature import VectorAssembler
Ohe_Assembler = VectorAssembler(inputCols = cat_ohe, outputCol = 'cat_features_assembled')
cat_data_ohe = Ohe_Assembler.transform(data_ohe)

#assemble all features
# all_features_assembler = VectorAssembler(inputCols=['cat_features_assembled','Var1','Var2','Var3','Var4','Var5',\
#                                           'Var6','Var7','Var8','NVVar1','NVVar2','NVVar3','NVVar4'],\
#                                 outputCol='features')
# all_data = all_features_assembler.transform(cat_data_ohe)

In [19]:
all_data

NameError: name 'all_data' is not defined

In [16]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator

t_data= all_data.select('features','Claim_Amount')

(trainingData, testData) = t_data.randomSplit([0.8, 0.2],47)


trainingData.cache()  
testData.cache()
print('Data preprocessing finished.')

lr = LinearRegression(featuresCol="features", labelCol="Claim_Amount")


In [17]:
import time
start = time.time()
print('Start training......')
lr_Model = lr.fit(trainingData)
lr_prediction = lr_Model.transform(testData)

evaluator = RegressionEvaluator(labelCol="Claim_Amount",\
                                predictionCol="prediction",\
                                metricName="rmse")

lr_rmse = evaluator.evaluate(lr_prediction)
end = time.time()
print('Execution time:',end-start)
print("RMSE = %g" % lr_rmse)

Execution time: 79.08290982246399
RMSE = 39.7016
